In [1]:
import os

from datetime import datetime
def time_now():
    '''Get Current Time'''
    
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S.%f")
    print("Current Time =", current_time)
    return now


############
print('Starting...')
start = time_now()
############

Starting...
Current Time = 18:29:11.522429


In [2]:
def create_folder_name(folder_prefix = 'NLP_RESULTS', file_time='datetime object'):
    file_name = f'{start.strftime("%d_%B_%Y_%Hh%Mm%Ss")}'
    file_name = f'{folder_prefix}_{file_name}'
    return file_name

In [3]:
batch_name = int(input('Enter Batch Number'))

Enter Batch Number 1


In [4]:
batch_name = f'batch_{str(batch_name).zfill(2)}'

from string import printable

printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [5]:
pos_tags = 'ADJ ADP ADV AUX CONJ CCONJ DET INTJ NOUN NUM PART PRON PROPN PUNCT SCONJ SYM VERB X SPACE'.split()

In [6]:
import pandas as pd

In [7]:
list_punct_ignore = ['"',  '!',  '–',  '.',  '‘',  ';',  '(',  ']',  ',',  '’',  '”',  '{',  ':',  ')',  '“',  '}',  '?',  '[']

In [8]:
pat_noun_fill_noun = [
    {'POS': {'IN': 'NOUN ADJ PROPN'.split()} , 'OP':'+'},
    {'POS': {'IN': 'ADP DET CCONJ'.split()} , 'OP':'+'},
    {'POS': {'IN': 'NOUN ADJ PROPN'.split()} , 'OP':'+'},
        ]

In [9]:
pat_noun = [
    {'POS': {'IN': 'NOUN ADJ PROPN PUNCT'.split()} , 'OP':'+', 'LEMMA':{'NOT_IN':list_punct_ignore}},
        ]

In [10]:
pat_noun

[{'POS': {'IN': ['NOUN', 'ADJ', 'PROPN', 'PUNCT']},
  'OP': '+',
  'LEMMA': {'NOT_IN': ['"',
    '!',
    '–',
    '.',
    '‘',
    ';',
    '(',
    ']',
    ',',
    '’',
    '”',
    '{',
    ':',
    ')',
    '“',
    '}',
    '?',
    '[']}}]

In [11]:
pat_verb_fill_noun = [
    {'POS': {'IN': 'VERB'.split()} , 'OP':'+'},
    {'POS': {'IN': 'ADP DET DET'.split()} , 'OP':'*'},
    {'POS': {'IN': 'NOUN ADJ PROPN'.split()} , 'OP':'+'},
        ]

In [12]:
pat_noun_fill_noun

[{'POS': {'IN': ['NOUN', 'ADJ', 'PROPN']}, 'OP': '+'},
 {'POS': {'IN': ['ADP', 'DET', 'CCONJ']}, 'OP': '+'},
 {'POS': {'IN': ['NOUN', 'ADJ', 'PROPN']}, 'OP': '+'}]

In [13]:
pat_noun_fill_verb = [
    {'POS': {'IN': 'NOUN ADJ PROPN'.split()} , 'OP':'+'},
    {'POS': {'IN': 'AUX'.split()} , 'OP':'*'},
    {'POS': {'IN': 'VERB'.split()} , 'OP':'+'},
        ]

In [14]:
#pat_list  = [eval(i) for i in dir() if "pat_" in i]

In [15]:
pat_list[0]

[{'POS': {'IN': ['NOUN', 'ADJ', 'PROPN', 'PUNCT']},
  'OP': '+',
  'LEMMA': {'NOT_IN': ['"',
    '!',
    '–',
    '.',
    '‘',
    ';',
    '(',
    ']',
    ',',
    '’',
    '”',
    '{',
    ':',
    ')',
    '“',
    '}',
    '?',
    '[']}}]

In [16]:
from itertools import chain
from collections import Counter

In [17]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [18]:
import regex as re
from spacy.matcher import Matcher

```PYTHON
with open('afc_statute_03.txt', encoding='utf-8') as afc_file:
    afc_stat_text = [line.strip() for line in afc_file.readlines()]
```

```PYTHON
#df = pd.read_pickle('AFC_CONGRESS_AKBAR_TRANSCRIPTS.pkl')
df = pd.DataFrame(data={
'TEXT': afc_stat_text
})
```

```PYTHON
df = df[df['TEXT'].apply(lambda x: len(x.split())>=3)].reset_index(drop=True)
```

In [19]:
#remove_this = list(dict(df['TEXT'].value_counts().head(5)))

In [20]:
#df = df[df['TEXT'].apply(lambda x: x not in remove_this)].reset_index(drop=True)

In [21]:
len(df)

NameError: name 'df' is not defined

In [22]:
# df['NLP'] = df['TEXT'].apply(lambda x: nlp(x))

In [23]:
#os.mkdir('nlps')

In [24]:
# df.to_pickle('./nlps/AFC_STATUTE_NLP.pkl', protocol=4)

# DIRECTORY OF NLP FILES


In [25]:
folder_name = f'nlps/'.replace('/', '')

In [26]:
try:
    threshold =  int(input('Enter Threshold Frequency:\n'))
except:
    threshold = 2

Enter Threshold Frequency:
 3


In [27]:
print(f'Threshold is {threshold}')

Threshold is 3


In [28]:
files_list = [i for i in os.listdir(folder_name) if i.endswith('pkl')]

In [29]:
files_list = [f'./{folder_name}/{i}' for i in files_list]

In [30]:
files_list

['./nlps/AFC_NLP.pkl', './nlps/AFC_STATUTE_NLP.pkl']

In [31]:
df_afc_trans, df_afc_stat  = pd.read_pickle(files_list[0]), pd.read_pickle(files_list[1])

In [32]:
df_afc_trans, df_afc_stat = df_afc_trans['NLP'.split()], df_afc_stat['NLP'.split()]

In [33]:
df_comb = pd.DataFrame(data = {'NLP': ['PLACE_HOLDER']})

In [34]:
df_comb = df_comb.append([df_afc_stat, df_afc_trans])
df_comb = df_comb[df_comb['NLP'] != 'PLACE_HOLDER'].reset_index(drop=True)

In [35]:
df = df_comb

In [36]:
def get_pos_lemma_pattern(doc, pat_name='anything', pat_collection='list_of_dictionaries'):
    '''
    Given a spacy doc object; find the span according to the pattern given.
    ''' 
    matcher =  Matcher(vocab = nlp.vocab)
    matcher.add(f'{pat_name}', pat_collection, greedy='LONGEST')
    
    
    doc_match = matcher(doc)
    
    
    list_container = []
    for match in doc_match:
        start = match[1]
        end = match[2]
        
        result_lemma = doc[start:end]
        result_lemma = [(i.lemma_ + i.whitespace_) for i in result_lemma] # new!
        result_lemma = ''.join(result_lemma).strip()

        result_token = doc[start:end]
        result_token = [(i.text + i.whitespace_) for i in result_token] # new!
        result_token = ''.join(result_token).strip()

        result_pos = doc[start:end]
        result_pos = [i.pos_ for i in result_pos] # new!
        result_pos = ' '.join(result_pos).strip()
        
        if len(result_lemma.split()) > 1:
            list_container.append((result_token, result_lemma, result_pos))

    return list_container

In [38]:
df

,NLP
0,"(STATUTES, OF, THE, ASIAN, FOOTBALL, CONFEDERA..."
1,"(In, interpreting, these, Statutes, ,, Regulat..."
2,"(1, ., "", Association, "", means, "", a, Footbal..."
3,"(2, ., "", AFC, "", means, the, "", Asian, Footba..."
4,"(3, ., "", Association, Football, "", means, "", ..."
...,...
1201,"(Sultan, as, well, as, all, of, tonight, 's, A..."
1202,"(round, of, applause, for, all, of, tonight, '..."
1203,"(taste, of, the, renowned, Armani, hospitality..."
1204,"(now, invite, you, to, join, us, in, the, foye..."


In [39]:
df['PAT_01'] = df['NLP'].apply(lambda x: get_pos_lemma_pattern(x, 'PAT_01', [pat_list[0]]))

In [40]:
df['PAT_02'] = df['NLP'].apply(lambda x: get_pos_lemma_pattern(x, 'PAT_02', [pat_list[1]]))

In [41]:
df['PAT_03'] = df['NLP'].apply(lambda x: get_pos_lemma_pattern(x, 'PAT_03', [pat_list[2]]))

In [42]:
df['PAT_04'] = df['NLP'].apply(lambda x: get_pos_lemma_pattern(x, 'PAT_04', [pat_list[3]]))

In [44]:
df

,NLP,PAT_01,PAT_02,PAT_03,PAT_04
0,"(STATUTES, OF, THE, ASIAN, FOOTBALL, CONFEDERA...","[(ASIAN FOOTBALL CONFEDERATION, ASIAN FOOTBALL...",[(STATUTES OF THE ASIAN FOOTBALL CONFEDERATION...,[],[]
1,"(In, interpreting, these, Statutes, ,, Regulat...","[(Standing Orders, Standing Orders, PROPN PROPN)]","[(Standing Orders of the Congress, Standing Or...","[(Regulations Governing, Regulations govern, P...","[(interpreting these Statutes, interpret these..."
2,"(1, ., "", Association, "", means, "", a, Footbal...","[(Football Association, Football Association, ...","[(evident from the context, evident from the c...","[(Football Association recognised, Football As...","[(recognised as such, recognise as such, VERB ..."
3,"(2, ., "", AFC, "", means, the, "", Asian, Footba...","[(Asian Football Confederation, Asian Football...",[],[],[]
4,"(3, ., "", Association, Football, "", means, "", ...","[(Association Football, Association Football, ...","[(Confederations and/or Member Associations, C...","[(game controlled, game control, NOUN VERB)]","[(controlled by FIFA, control by FIFA, VERB AD..."
...,...,...,...,...,...
1201,"(Sultan, as, well, as, all, of, tonight, 's, A...","[(Award winners, Award winner, PROPN NOUN), (g...","[(Award winners for a group photograph, Award ...",[],"[(have the winners, have the winner, VERB DET ..."
1202,"(round, of, applause, for, all, of, tonight, '...","[(AFC annual awards, AFC annual award, PROPN A...","[(guests ladies and gentlemen, guest lady and ...","[(AFC annual awards muscat, AFC annual award m...","[(concludes the AFC annual awards, conclude th..."
1203,"(taste, of, the, renowned, Armani, hospitality...","[(good night good night, good night good night...",[(general secretaries of all the AFC member as...,"[(pleasure co-hosting, pleasure co-host, NOUN ...","[(recognize the true Diamonds, recognize the t..."
1204,"(now, invite, you, to, join, us, in, the, foye...","[(post event reception, post event reception, ...",[],[],"[(have a good evening, have a good evening, VE..."


In [43]:
#df.to_pickle('COMBINED_STAT_TRANS_AFC_PATTERNS_10_05_2022_STAGE_01.pkl', protocol=4)

In [45]:
df['PAT_01'].iloc[4][1]

('Member Associations', 'Member Associations', 'PROPN PROPN')

In [46]:
dict_pat_01 = dict(Counter(list(chain(*list(df['PAT_01'].apply(lambda x: [i[1].title() for i in x]))))).most_common(200))

In [47]:
dict_pat_02 = dict(Counter(list(chain(*list(df['PAT_02'].apply(lambda x: [i[1].title() for i in x]))))).most_common(200))

In [48]:
dict_pat_03 = dict(Counter(list(chain(*list(df['PAT_03'].apply(lambda x: [i[1].title() for i in x]))))).most_common(200))

In [49]:
dict_pat_04 = dict(Counter(list(chain(*list(df['PAT_04'].apply(lambda x: [i[1].title() for i in x]))))).most_common(200))

In [50]:
dict_pat_01 = {k:v for k,v in dict_pat_01.items() if v>1}

In [51]:
dict_pat_02 = {k:v for k,v in dict_pat_02.items() if v>1}

In [52]:
dict_pat_03 = {k:v for k,v in dict_pat_03.items() if v>1}

In [53]:
dict_pat_04 = {k:v for k,v in dict_pat_04.items() if v>1}

In [54]:
[len(eval(i)) for i in dir() if 'dict_pat_0' in i]

[200, 200, 166, 200]

In [87]:
df_01 = pd.DataFrame(data = list(zip(dict_pat_01.keys(), dict_pat_01.values())), columns='TERMS FREQ'.split())
df_02 = pd.DataFrame(data = list(zip(dict_pat_02.keys(), dict_pat_02.values())), columns='TERMS FREQ'.split())
df_03 = pd.DataFrame(data = list(zip(dict_pat_03.keys(), dict_pat_03.values())), columns='TERMS FREQ'.split())
df_04 = pd.DataFrame(data = list(zip(dict_pat_04.keys(), dict_pat_04.values())), columns='TERMS FREQ'.split())

In [88]:
x1 = df_01['FREQ'].mean(), df_01['FREQ'].max(), (df_01['FREQ'].mean() + df_01['FREQ'].max())/2
x2 = df_02['FREQ'].mean(), df_02['FREQ'].max(), (df_02['FREQ'].mean() + df_02['FREQ'].max())/2
x3 = df_03['FREQ'].mean(), df_03['FREQ'].max(), (df_03['FREQ'].mean() + df_03['FREQ'].max())/2
x4 = df_04['FREQ'].mean(), df_04['FREQ'].max(), (df_04['FREQ'].mean() + df_04['FREQ'].max())/2

In [89]:
x1, x2, x3, x4

((11.015, 196, 103.5075),
 (3.485, 19, 11.2425),
 (3.7228915662650603, 34, 18.86144578313253),
 (4.165, 29, 16.5825))

In [97]:
def repeat(term, freq, tup):
    if freq >= int(round(tup[2])):
        return f"""
{term}\t\t\t\t{term}

------------------------------------------------------------------
{term}\t\t\t\t{term}

------------------------------------------------------------------
{term}\t\t\t\t{term}

------------------------------------------------------------------
""".strip()
    elif int(round(tup[0])) <= freq:
        return f"""
{term}\t\t\t\t{term}

------------------------------------------------------------------
{term}\t\t\t\t{term}

------------------------------------------------------------------
""".strip()
    else:
        return f"""
{term}\t\t\t\t{term}

------------------------------------------------------------------
""".strip()

In [98]:
df_01['WRITE'] = df_01.apply(lambda x: repeat(x['TERMS'], x['FREQ'], x1), axis=1)
df_02['WRITE'] = df_02.apply(lambda x: repeat(x['TERMS'], x['FREQ'], x2), axis=1)
df_03['WRITE'] = df_03.apply(lambda x: repeat(x['TERMS'], x['FREQ'], x3), axis=1)
df_04['WRITE'] = df_04.apply(lambda x: repeat(x['TERMS'], x['FREQ'], x4), axis=1)

In [102]:
# df_01['WRITE'.split()].to_csv('PAT_01.txt', encoding='utf-8', header=False, index=False)
# df_02['WRITE'.split()].to_csv('PAT_02.txt', encoding='utf-8', header=False, index=False)
# df_03['WRITE'.split()].to_csv('PAT_03.txt', encoding='utf-8', header=False, index=False)
# df_04['WRITE'.split()].to_csv('PAT_04.txt', encoding='utf-8', header=False, index=False)

,TERMS,FREQ
0,Guest Lady And Gentleman,19
1,Press The Submit Button,15
2,Vote The Voting Window,15
3,Amendment To The Statute,12
4,Afc Organisation Regulations And Relevant Regu...,12
...,...,...
195,Window For Voting,2
196,Moment Of Silence,2
197,Administrator And Fan,2
198,Article Of The Afc Statute,2


In [57]:
list_dict_patterns = sorted([i for i in dir() if 'dict_pat_0' in i])

In [58]:
list_dict_patterns

['dict_pat_01', 'dict_pat_02', 'dict_pat_03', 'dict_pat_04']

In [59]:
list_expressions_dfs = [f"""
df_AFC_STAT_{str(idx+1).zfill(2)} = pd.DataFrame(data= {{
'LEMMA':{i}.keys(),
'FREQ': {i}.values()
}}).to_pickle('COMB_PAT_{str(idx+1).zfill(2)}.pkl', protocol=4)
""".strip() for (idx,i) in enumerate(list_dict_patterns)]

In [60]:
for i in list_expressions_dfs:
    print(i)
    print('-'*50)
    exec(i)

df_AFC_STAT_01 = pd.DataFrame(data= {
'LEMMA':dict_pat_01.keys(),
'FREQ': dict_pat_01.values()
}).to_pickle('COMB_PAT_01.pkl', protocol=4)
--------------------------------------------------
df_AFC_STAT_02 = pd.DataFrame(data= {
'LEMMA':dict_pat_02.keys(),
'FREQ': dict_pat_02.values()
}).to_pickle('COMB_PAT_02.pkl', protocol=4)
--------------------------------------------------
df_AFC_STAT_03 = pd.DataFrame(data= {
'LEMMA':dict_pat_03.keys(),
'FREQ': dict_pat_03.values()
}).to_pickle('COMB_PAT_03.pkl', protocol=4)
--------------------------------------------------
df_AFC_STAT_04 = pd.DataFrame(data= {
'LEMMA':dict_pat_04.keys(),
'FREQ': dict_pat_04.values()
}).to_pickle('COMB_PAT_04.pkl', protocol=4)
--------------------------------------------------


In [ ]:
# for i in list_expressions_dfs:
#     exec(i)